Report on runtime concurrency for kaggle simulations

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
META ="../input/meta-kaggle/"

COMP = 'lux-ai-2021' # alternative report
#COMP = 'google-football' # this report

COMPETITIONS = {
    'lux-ai-2021': 30067,
    'hungry-geese': 25401,
    'rock-paper-scissors': 22838,
    'santa-2020': 24539,
    'halite': 18011,
    'google-football': 21723
}

In [ ]:
episodes_df = pd.read_csv(META + "Episodes.csv")
episodes_df = episodes_df[episodes_df.CompetitionId == COMPETITIONS[COMP]] # lux
episodes_df = episodes_df.set_index(['Id'])
episodes_df['CreateTime'] = pd.to_datetime(episodes_df['CreateTime'], format="%m/%d/%Y %H:%M:%S")
episodes_df['EndTime'] = pd.to_datetime(episodes_df['EndTime'], format="%m/%d/%Y %H:%M:%S")
episodes_df['RunTime'] = (episodes_df.EndTime-episodes_df.CreateTime) / pd.Timedelta(seconds=1)
meantime = episodes_df['RunTime'].mean()
print(f'Average match time is {int(meantime)} seconds')

In [ ]:
starts = pd.Series(1, episodes_df['CreateTime'])
ends = pd.Series(-1, episodes_df['EndTime'] + pd.Timedelta('1 sec'))

min_time = episodes_df['CreateTime'].min()
max_time = episodes_df['EndTime'].max()
ts_index = pd.date_range(min_time, max_time, freq = 's')

ts_changes = pd.concat([starts, ends]).groupby(level=0).sum().cumsum()
concurrency = ts_changes.reindex(ts_index, method='ffill')

In [ ]:
concurrency.resample(f'{int(meantime)}S').max()

In [ ]:
plt.figure(figsize=(10, 10))
number_of_windows = 5
concurrency.resample(f'{int(number_of_windows*meantime)}S').max().plot(style='k--', label='Series')
plt.legend()